# Data preprocessing: Feature Engineering

**NOTEBOOK GOAL**: Extracting mean store sales and mean sales per region

**DATASET TRANSFORMATION**: `preprocessed2_OHE_imputation_train.csv` >> `preprocessed_train.csv`

**INDEX**

- [Deletion of IsOpen = 0 rows](#Deletion-of-IsOpen-=-0-rows)
- [MeanStoreSales feature](#MeanStoreSales-feature)
- [MeanReagionSales feature](#MeanReagionSales feature)
- [Day of the Week considering days adjacency](#Day-of-the-Week-considering-days-adjacency) 

In [17]:
import numpy as np 
import pandas as pd

features_to_print = ['StoreID', 'Date']

In [18]:
df = pd.read_csv("dataset/preprocessed2_OHE_train.csv")

### Deletion of IsOpen = 0 rows

Let's delete the rows of store closed.

In [19]:
df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,StoreType_StandardMarket,StoreType_ShoppingCenter,AssortmentType_General,AssortmentType_WithNFDept,AssortmentType_WithFishDept,Events_Fog,Events_Hail,Events_Rain,Events_Snow,Events_Thunderstorm
0,1000,01/03/2016,0,1,0,Hyper Market,General,326,7,495,...,0,0,1,0,0,0,0,1,1,0
1,1000,02/03/2016,0,1,0,Hyper Market,General,326,7,608,...,0,0,1,0,0,0,0,0,1,0
2,1000,04/03/2016,0,1,0,Hyper Market,General,326,7,665,...,0,0,1,0,0,0,0,1,0,0
3,1000,05/03/2016,0,1,0,Hyper Market,General,326,7,630,...,0,0,1,0,0,0,0,0,0,0
4,1000,06/03/2016,0,0,0,Hyper Market,General,326,7,0,...,0,0,1,0,0,0,0,0,0,0


In [20]:
df = df[df.IsOpen==1]

In [21]:
df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,StoreType_StandardMarket,StoreType_ShoppingCenter,AssortmentType_General,AssortmentType_WithNFDept,AssortmentType_WithFishDept,Events_Fog,Events_Hail,Events_Rain,Events_Snow,Events_Thunderstorm
0,1000,01/03/2016,0,1,0,Hyper Market,General,326,7,495,...,0,0,1,0,0,0,0,1,1,0
1,1000,02/03/2016,0,1,0,Hyper Market,General,326,7,608,...,0,0,1,0,0,0,0,0,1,0
2,1000,04/03/2016,0,1,0,Hyper Market,General,326,7,665,...,0,0,1,0,0,0,0,1,0,0
3,1000,05/03/2016,0,1,0,Hyper Market,General,326,7,630,...,0,0,1,0,0,0,0,0,0,0
5,1000,07/03/2016,0,1,1,Hyper Market,General,326,7,763,...,0,0,1,0,0,0,0,0,0,0


In [22]:
df.drop(['IsOpen'], axis=1, inplace=True)

### MeanStoreSales feature

Add the mean of the sales of the store

In [23]:
df_mean = df.groupby('StoreID')['NumberOfSales'].mean()
def mean_sales(value):
    # since stores ID are from 1000 to 1749 let's subtract 1000
    return df_mean.iloc[value-1000]

df['MeanStoreSales'] = df.StoreID.apply(mean_sales)

In [24]:
print(df_mean.shape)
df_mean.head()

(749,)


StoreID
1000    7675.446488
1001    3154.465753
1002    4968.047776
1003    5409.347107
1004    4046.109865
Name: NumberOfSales, dtype: float64

### MeanReagionSales feature

Add the mean of sales for the region (regardless of the store)

In [25]:
df_mean_reg = df.groupby('Region')['NumberOfSales'].mean()

In [26]:
def mean_sales_region(value):
    return df_mean_reg.iloc[value]

df['MeanRegionSales'] = df.Region.apply(mean_sales_region)

In [27]:
features_to_print += ['D_DayOfweek', 'MeanStoreSales', 'MeanRegionSales']

df[df.Date=="01/03/2016"][features_to_print].head().T

,0,729,1458,2187,2916
StoreID,1000,1001,1002,1003,1004
Date,01/03/2016,01/03/2016,01/03/2016,01/03/2016,01/03/2016
D_DayOfweek,1,1,1,1,1
MeanStoreSales,7675.45,3154.47,4968.05,5409.35,4046.11
MeanRegionSales,4702.75,4885.4,5839.41,4702.75,4679.27


### MeanCustomers feature

Add the mean of the sales of the store

In [28]:
df_mean = df.groupby('StoreID')['NumberOfCustomers'].mean()
def mean_cust(value):
    # since stores ID are from 1000 to 1749 let's subtract 1000
    return df_mean.iloc[value-1000]

df['MeanCustomers'] = df.StoreID.apply(mean_cust)

In [29]:
print(df_mean.shape)
df_mean.head()

(749,)


StoreID
1000    622.769231
1001    194.358121
1002    379.586491
1003    247.581818
1004    203.199552
Name: NumberOfCustomers, dtype: float64

### MeanCustomers feature

Add the mean of sales for the region (regardless of the store)

In [30]:
df_mean_reg = df.groupby('Region')['NumberOfCustomers'].mean()

In [31]:
def mean_cust_region(value):
    return df_mean_reg.iloc[value]

df['MeanRegionCustomers'] = df.Region.apply(mean_cust_region)

In [32]:
features_to_print += ['D_DayOfweek', 'MeanStoreSales', 'MeanRegionSales','MeanCustomers', 'MeanRegionCustomers']

df[df.Date=="01/03/2016"][features_to_print].head().T

,0,729,1458,2187,2916
StoreID,1000,1001,1002,1003,1004
Date,01/03/2016,01/03/2016,01/03/2016,01/03/2016,01/03/2016
D_DayOfweek,1,1,1,1,1
MeanStoreSales,7675.45,3154.47,4968.05,5409.35,4046.11
MeanRegionSales,4702.75,4885.4,5839.41,4702.75,4679.27
D_DayOfweek,1,1,1,1,1
MeanStoreSales,7675.45,3154.47,4968.05,5409.35,4046.11
MeanRegionSales,4702.75,4885.4,5839.41,4702.75,4679.27
MeanCustomers,622.769,194.358,379.586,247.582,203.2
MeanRegionCustomers,284.209,316.367,406.117,284.209,258.535


### Day of the Week considering days adjacency

<https://www.reddit.com/r/MachineLearning/comments/2hzuj5/how_do_i_encode_day_of_the_week_as_a_predictor/>

In [33]:
sorted(df['D_DayOfweek'].unique())

[0, 1, 2, 3, 4, 5, 6]

In [34]:
from math import pi, cos, sin

# angle in rad
df['rad_ang'] = (df['D_DayOfweek'] / 7) * (2 * pi)
#df['D_DayOfWeek_cos'] = df.apply(lambda x: cos(x['rad_ang']), axis=1)
#df['D_DayOfWeek_sin'] = df.apply(lambda x: sin(x['rad_ang']), axis=1)
df['D_DayOfWeek_cos'] = df['rad_ang'].apply(lambda x: cos(x))
df['D_DayOfWeek_sin'] = df['rad_ang'].apply(lambda x: sin(x))

features_to_print  += ['rad_ang', 'D_DayOfWeek_cos', 'D_DayOfWeek_sin']
df[features_to_print].head(20).T

,0,1,2,3,5,6,7,8,9,10,12,13,14,15,16,17,19,20,21,22
StoreID,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
Date,01/03/2016,02/03/2016,04/03/2016,05/03/2016,07/03/2016,08/03/2016,09/03/2016,10/03/2016,11/03/2016,12/03/2016,14/03/2016,15/03/2016,16/03/2016,17/03/2016,18/03/2016,19/03/2016,21/03/2016,22/03/2016,23/03/2016,24/03/2016
D_DayOfweek,1,2,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3
MeanStoreSales,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45
MeanRegionSales,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75
D_DayOfweek,1,2,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3
MeanStoreSales,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45,7675.45
MeanRegionSales,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75,4702.75
MeanCustomers,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769,622.769
MeanRegionCustomers,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209,284.209


In [35]:
# let's drop the angle column since it has the same menaing of the day of week
df.drop('rad_ang', inplace=True, axis=1)
df.head(1).T

,0
StoreID,1000
Date,01/03/2016
IsHoliday,0
HasPromotions,0
StoreType,Hyper Market
AssortmentType,General
NearestCompetitor,326
Region,7
NumberOfCustomers,495
NumberOfSales,5676


## Write to file

In [36]:
df.to_csv('./dataset/preprocessed_train.csv', index=False)

## Additional sales of Store per month (tested and was not successful)

In [37]:
break_here_exec

#dfmean = df.groupby(['StoreID','D_Month'])['NumberOfSales'].mean()
dfmean.head().T

NameError: name 'break_here_exec' is not defined

In [ ]:
dfmean

In [ ]:
dfmean = dfmean.reset_index()
dfmean.head().T

In [ ]:
store=1003
m=5
dfmean[(dfmean.StoreID ==store) & (dfmean.D_Month ==m)]['NumberOfSales'][(store-1000)*12+m-1]

In [ ]:
df['Mean_month_sale']

In [ ]:
df.loc[:10,'Mean_month_sale']=dfmean[(dfmean.StoreID ==1000) & (dfmean.D_Month ==1)]['NumberOfSales'][0]

In [ ]:
store=1001
month=3
df.loc[(df.StoreID ==store) & (df.D_Month ==month),'Mean_month_sale']=\
dfmean[(dfmean.StoreID ==store) & (dfmean.D_Month ==month)]['NumberOfSales'][(store-1000)*12+month-1]

In [ ]:
for store in (1000,1748):
    for month in (1,12):
        df.loc[(df.StoreID ==store) & (df.D_Month ==month),'Mean_month_sale']=\
                dfmean[(dfmean.StoreID ==store) & (dfmean.D_Month ==month)]['NumberOfSales'][(store-1000)*12+month-1]

In [ ]:
#def mean_sales_month(store,month):  #takes too much time
#    # since stores ID are from 1000 to 1749 let's subtract 1000
#    return dfmean[(dfmean.StoreID ==store) & (dfmean.D_Month ==month)]